In [4]:
import pandas as pd
import numpy as np
import requests
import re
import ollama
import random
import math
from tiktoken import get_encoding

In [20]:
labels_with_definitions = [
    ("Peso", "Indica la ligereza de la zapatilla, generalmente expresado en gramos. El peso puede influir en el rendimiento y la comodidad durante la carrera."),
    ("Material del upper (parte superior)", "Describe los materiales utilizados en la parte superior de la zapatilla, como malla, cuero sintético o tejidos técnicos, que afectan la transpirabilidad, flexibilidad y soporte."),
    ("Material de la mediasuela", "Se refiere a los compuestos empleados en la entresuela, como espumas EVA o tecnologías propietarias, que proporcionan amortiguación y absorción de impactos."),
    ("Suela exterior", "Detalla el tipo de goma o caucho utilizado en la suela y el diseño del patrón de tracción, aspectos que influyen en el agarre y la durabilidad en diversas superficies."),
    ("Sistema de amortiguación", "Especifica las tecnologías o materiales destinados a reducir el impacto durante la pisada, contribuyendo al confort y la protección de las articulaciones."),
    ("Drop (diferencial talón-punta)", "Indica la diferencia de altura entre el talón y la punta de la zapatilla, medida en milímetros. Un drop alto suele ofrecer mayor amortiguación en el talón, mientras que un drop bajo promueve una pisada más natural."),
    ("Tipo de pisada", "Clasifica la zapatilla según su adecuación para diferentes tipos de pisada: neutra, pronadora o supinadora. Esto es esencial para elegir un calzado que se adapte a la biomecánica del corredor."),
    ("Tipo de uso", "Define el propósito principal de la zapatilla, como entrenamiento diario, competición, trail running o uso casual, lo que orienta sobre su diseño y funcionalidades específicas."),
    ("Género", "Indica si la zapatilla está diseñada para hombres, mujeres o es un modelo unisex, considerando diferencias anatómicas y de tamaño."),
    ("Tallas disponibles", "Especifica el rango de tallas en las que se ofrece la zapatilla, asegurando que el corredor pueda encontrar un ajuste adecuado."),
    ("Anchura", "Algunas marcas ofrecen diferentes anchos (estrecho, estándar, ancho) para adaptarse a diversas morfologías del pie."),
    ("Precio", "Proporciona el costo de la zapatilla, un factor determinante en la decisión de compra."),
    ("Tecnologías adicionales", "Incluye características especiales como impermeabilidad, reflectividad, sistemas de ajuste personalizados o elementos de estabilidad que mejoran la funcionalidad del calzado."),
]

## Funciones

In [5]:
def transformar_texto(texto, marca):
    if texto is None or (isinstance(texto, float) and np.isnan(texto)):
        return texto
    
    if marca.lower() == "adidas":
        # Transformación original para adidas
        if isinstance(texto, (list, np.ndarray)):
            texto = ", ".join(map(str, texto))
        else:
            texto = str(texto)
        texto = texto.strip("[]")
        texto = re.sub(r",\s*", '} {', texto)
        texto = '{' + texto + '}'
        return texto
    
    elif marca.lower() == "nike":
        # Transformación específica para nike
        if isinstance(texto, list):
            # Elimina claves con valores irrelevantes
            texto_limpio = [
                {k.strip(): v.strip() for k, v in d.items() if k.strip() not in ['\xa0']}
                for d in texto
                if isinstance(d, dict)
            ]
            # Filtra elementos vacíos o irrelevantes
            texto_limpio = [d for d in texto_limpio if d]
            return texto_limpio
        return texto  # Si no es lista, regresa el texto original

    else:
        raise ValueError(f"Marca '{marca}' no reconocida. Usa 'adidas' o 'nike'.")


In [6]:
def obtener_respuesta_ollama(prompt):
    response = ollama.chat(
        model="llama3.2:latest",
        messages = [
           {
               "role":"user",
               "content": prompt
           } 
        ]
    )
    # La respuesta es un generador; concatenamos las partes
    return response

In [ ]:



def generate_prompt_ollama(product, labels_with_definitions):
    labels = [label for label, _ in labels_with_definitions]
    prompt = f"""
        You are an assistant specialized in extracting structured information from product descriptions and organizing it into tables.
        Your task is to extract the following information from the product details and label it according to the provided labels: {', '.join(labels)}.
        Each label has the following definition to help guide your extraction:

        {''.join([f'- {label}: {definition}\n' for label, definition in labels_with_definitions])}

        If a label does not have a clear match in the details, complete its value with "null".

        Product information:
        - Details: {product['details']}
        - Description: {product['description']}
        - Category: {product['category']}

        Provide the information in a table with columns corresponding to each label. 
        The table must include **two complete rows**:
        1. The first row contains the label names.
        2. The second row contains the corresponding labeled values.

        Expected response format:
        | {' | '.join(labels)} |
        | {' | '.join(['---'] * len(labels))} |
        | value_1 | value_2 | ... |
        
        Example:
        If the labels are "details", "description", and "category", and the extracted values are 
        "Comfortable sneakers", "Made with recycled materials", and "Footwear", respectively, 
        your response should be:

        | details | description | category |
        | --- | --- | --- |
        | Comfortable sneakers | Made with recycled materials | Footwear |

        Remember:
        - The response must contain two complete rows.
        - Only respond with the table and **do not include additional text**.

        Now, extract and structure the information for the provided product:

        | {' | '.join(labels)} |
        | {' | '.join(['---'] * len(labels))} |
        |"""
    return prompt.strip()


In [8]:
# Tokenizer function (using tiktoken for GPT-like models)
def count_tokens(text):
    try:
        encoding = get_encoding("cl100k_base")  # Example encoding for GPT-like models
        return len(encoding.encode(text))
    except Exception:
        return len(text.split())  # Fallback: approximate by word count

In [39]:
# Función de simulación de Monte Carlo corregida
def monte_carlo_simulation(products, models, iterations=1000, num_products=None):
    results = {}
    
    for model_name, model_info in models.items():
        tokens_per_product = {}
        costs_per_product = {}
        
        for _ in range(iterations):
            # Muestra una fracción de los productos si se especifica
            if num_products is not None and num_products < len(products):
                sampled_products = random.sample(products, num_products)
            else:
                sampled_products = products

            for product in sampled_products:
                product_id = product.get('id', id(product))  # Usamos un identificador único para cada producto
                prompt = generate_prompt_ollama(product, labels_with_definitions)
                tokens = count_tokens(prompt)
                
                # Verifica si el número de tokens excede la ventana de contexto
                if tokens > model_info['context_window']*1000:
                    print(f"Advertencia: El prompt excede la ventana de contexto para el modelo {model_name}")
                    # Puedes manejar este caso según necesites (e.g., omitir, truncar)
                    continue
                
                cost = (tokens / 1000) * model_info['cost_in']
                
                # Actualiza los máximos y mínimos por producto
                if product_id not in tokens_per_product:
                    tokens_per_product[product_id] = {'max': tokens, 'min': tokens}
                    costs_per_product[product_id] = {'max': cost, 'min': cost}
                else:
                    tokens_per_product[product_id]['max'] = max(tokens_per_product[product_id]['max'], tokens)
                    tokens_per_product[product_id]['min'] = min(tokens_per_product[product_id]['min'], tokens)
                    costs_per_product[product_id]['max'] = max(costs_per_product[product_id]['max'], cost)
                    costs_per_product[product_id]['min'] = min(costs_per_product[product_id]['min'], cost)
        
        # Después de todas las iteraciones, obtenemos los máximos y mínimos globales
        max_tokens = max([data['max'] for data in tokens_per_product.values()], default=0)
        min_tokens = min([data['min'] for data in tokens_per_product.values()], default=0)
        max_cost = max([data['max'] for data in costs_per_product.values()], default=0)
        min_cost = min([data['min'] for data in costs_per_product.values()], default=0)
        
        results[model_name] = {
            'max_tokens': max_tokens,
            'min_tokens': min_tokens,
            'max_cost': max_cost,
            'min_cost': min_cost,
        }
    return results

## Lectura de datos

In [10]:
url ="https://scraping-firestore-178159629911.us-central1.run.app//v1/scraping/"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(response.status_code)
else:
    print(f'Error: {response.status_code}')

df_raw = pd.DataFrame(data)

200


## Código

In [13]:
df_adidas = df_raw[df_raw['store'] == 'adidas']

In [14]:
# Lista de descripciones de productos
df_adidas['details_transformado'] = df_adidas['details'].apply(lambda x: transformar_texto(x, 'adidas'))

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3872\354877031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adidas['details_transformado'] = df_adidas['details'].apply(lambda x: transformar_texto(x, 'adidas'))


In [15]:
# Crear un diccionario con las llaves deseadas desde todo el DataFrame
productos = [
    {
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category']
    }
    for _, row in df_adidas.iterrows()  # Recorre todo el DataFrame
    if row['details_transformado'] != '{}'  # Filtra donde los detalles no estén vacíos
]

In [16]:
# Model configurations
models = {
    "jamba15large": {'cost_in': 0.002, 'cost_out': 0.008, 'accuracy': 64, 'context_window': 256},
    "claude35sonnet": {'cost_in': 0.003, 'cost_out': 0.015, 'accuracy': 80, 'context_window': 200},
    "llama3170b": {'cost_in': 0.00099, 'cost_out': 0.00099, 'accuracy': 66, 'context_window': 128},
    "mistrallarge": {'cost_in': 0.004, 'cost_out': 0.012, 'accuracy': 56, 'context_window': 33},
}

In [41]:
# Ahora puedes especificar el número de productos a procesar (por ejemplo, 10)
num_products_to_process = 100  # O usa len(productos) para procesar todos

# Ejecutar la simulación de Monte Carlo
simulation_results = monte_carlo_simulation(productos, models, iterations=1000, num_products=num_products_to_process)

# Mostrar los resultados
import pandas as pd
df_results = pd.DataFrame(simulation_results).transpose().reset_index()
df_results.columns = ['Model', 'Max Tokens', 'Min Tokens', 'Max Cost', 'Min Cost']


In [42]:
print(df_results)


            Model  Max Tokens  Min Tokens  Max Cost  Min Cost
0    jamba15large      1787.0      1303.0  0.003574  0.002606
1  claude35sonnet      1787.0      1303.0  0.005361  0.003909
2      llama3170b      1787.0      1303.0  0.001769  0.001290
3    mistrallarge      1787.0      1303.0  0.007148  0.005212


In [43]:
df_adidas.shape

(519, 14)

In [24]:
# Crear un diccionario con las llaves deseadas
productos = [
    {
        "details": row['details_transformado'],
        "description": row['description'],
        "category": row['category']
    }
    for _, row in df_adidas[:10].iterrows() 
    if row['details_transformado'] != '{}'
]

In [27]:
dfs = []
for producto in productos:
    prompt = generate_prompt_ollama(producto, labels_with_definitions)
    respuesta = obtener_respuesta_ollama(prompt)
    print(producto)
    print("Respuesta del modelo:")
    print(respuesta["message"]["content"])
    print("\n------------------------\n")


{'details': '{Horma clásica} {Parte superior sintética} {Forro interno textil} {Varillas ENERGYRODS 2.0 que limitan la pérdida de energía} {Amortiguación Lightstrike Pro} {Peso: 183 g (talla CO 37} {5)} {Caída mediasuela: 6 mm (talón: 38 mm / antepié: 32 mm)} {La parte superior contiene al menos un 50 % de material reciclado} {Color del artículo: Pink Spark / Aurora Met. / Sandy Pink} {Número de artículo: ID3612}', 'description': "Los Adizero Adios Pro 3 son la máxima expresión de los productos Adizero Racing. Fueron diseñados con y para atletas para lograr hazañas increíbles. Estos tenis de running adidas están diseñados para optimizar la eficiencia del running. Nuestras varillas ENERGYRODS 2.0 de carbono ofrecen ligereza y firmeza para pasos ágiles y eficientes. La tecnología LIGHTSTRIKE PRO ultraliviana amortigua cada paso con las tres capas de espuma resistente que te ayudan a mantener la energía a largo plazo. Todo sobre una delgada suela de caucho textil para un agarre extraordin